In [1]:
import os

In [2]:
%pwd

'/home/ehsan/PycharmProjects/TextSummarizer/Text-Summarizer/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/ehsan/PycharmProjects/TextSummarizer/Text-Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    



In [6]:
from textSummarizer.constants import *

from textSummarizer.utils.common import read_yaml, create_dirs


In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath= CONFIG_FILE_PATH,
        params_filepath= PARAMS_FILE_PATH):

        self.config = read_yaml(Path(config_filepath))
        self.params = read_yaml(Path(params_filepath))

        create_dirs([self.config.artifacts_root], True)

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_dirs([config.root_dir], True)

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_data_dir
        )

        return data_ingestion_config
    
    

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config:DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename= self.config.local_data_file
            )
            logger.info(f'{filename} downloaded! with following headers: \n{headers}')

        else: 
            logger.info(f'file already exists! with size: {get_size(Path(self.config.local_data_file))}')


    def exract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_file:
            zip_file.extractall(unzip_path)

In [10]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.exract_zip_file()

except Exception as e:
    raise e

[2024-04-04 02:35:39,034: INFO: common:  Loaded config/config.yaml]
[2024-04-04 02:35:39,038: INFO: common:  Loaded params.yaml]
[2024-04-04 02:35:39,040: INFO: common:  create directory at artifacts]
[2024-04-04 02:35:39,041: INFO: common:  create directory at artifacts/data_ingestion]
[2024-04-04 02:35:59,157: INFO: 2432937049:  artifacts/data_ingestion/data.zip downloaded! with following headers: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: EDE0:3F20D2:48F3FBD:4C2EB3D:660DE0C4
Accept-Ranges: bytes
Date: Wed, 03 Apr 2024 23:05:42 GMT
Via: 1.1 varnish
X-Served-By: cache-lcy-eglc8600078-LCY
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S171